In [14]:
import tabula
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
    
files = {
    2014: {
        "1-sem": {
            "theory": "imt-data/Taxasdeaprovacao_1_Semestre_2014_Teoricas_Integradas.pdf",
            "driving": "imt-data/Taxasdeaprovacao_1_Semestre_2014_Praticas_Integradas.pdf"
        },
        "2-sem": {
            "theory": "imt-data/Taxasdeaprovacao_2_Semestre_2014_Teoricas_Integradas.pdf",
            "driving": "imt-data/Taxasdeaprovacao_2_Semestre_2014_Praticas_Integradas.pdf"
        }
    },    
    2015: "imt-data/TaxasApr_2015_Relatorio_Escolas_de_Condução.pdf",
    2016: "imt-data/TaxasApr_2016_Relatorio_EscolasDeConducao.pdf",
    2017: "imt-data/TaxasApr_2017_Relatorio_EscolasDeConducao.pdf",
    2018: "imt-data/TaxasApr_2018_Relatorio_EscolasDeConducao.pdf",
    2019: "imt-data/EscolasdeCondução-2019.pdf",
    2020: "imt-data/EscolasdeCondução-2020.pdf"
}

def parse_rate (x):
    
    if x == '#DIV/0!':
        return np.NAN
    elif pd.isna(x):
        return x
    else:
        return float((x.replace(',', '.').replace('%', 'e-2')))
    
def parse_int (x):
    
    if pd.isna(x):
        return 
    else:
        return np.Int64(x)
    
def name_to_keyword(x):
    
    s = unidecode(x)
    
    s = re.sub(r'[^a-zA-Z0-9\s]', '', s)
    
    s = s.lower() \
    .replace('escola de conducao da ', '') \
    .replace('escola de conducao de ', '') \
    .replace('escola de conducao ', '') \
    .replace('escola do ', '') \
    .strip() \
    .replace('  ', ' ') \
    .replace(' ', '-')
    
    return s

def nec_to_string (s): 
    
    if pd.isna(s):
        return "xxxxx"
    else:
        return str(s).zfill(5)


headers = ['nec', 'name_raw', 't_scheduled', 't_done', 't_rate', 'd_scheduled', 'd_done', 'd_rate']
headers_full = ['nec', 'name_raw', 't_scheduled', 't_done', 't_rate', 'd_scheduled', 'd_done', 'd_rate', 'total_scheduled', 'total_done', 'total_rate']

In [15]:
year=2015
path = files[year]
all_pages = tabula.read_pdf(path, stream=True, pages="all", pandas_options={'header': None})
T = pd.concat(all_pages, ignore_index=True, sort=False)

T.columns = headers_full

T["nec"] = T["nec"].astype('Int64')
T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)

#quality control
summary = T.tail(1) 
T=T.iloc[:-1,:]

T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)
T2015 = T.set_index('k')
summary2015 = summary

#####################################

assert(summary2015["d_done"].values[0] == T2015["d_done"].sum())
assert(summary2015["t_done"].values[0] == T2015["t_done"].sum())
assert(summary2015["d_scheduled"].values[0] == T2015["d_scheduled"].sum())
assert(summary2015["t_scheduled"].values[0] == T2015["t_scheduled"].sum())

T2015[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2015[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2015['total_passed'] = T2015['d_done'].multiply(T2015['d_rate'], fill_value = 1) + T2015['t_done'].multiply(T2015['t_rate'], fill_value =1)
total_passed = T2015['total_passed'].sum() / (T2015["d_done"].sum() + T2015["t_done"].sum())
assert(round(total_passed, 4) == summary2015["total_rate"].values[0])

In [16]:
year=2016
path = files[year]

all_pages = tabula.read_pdf(path, stream=True, pages="all", pandas_options={'header': None})
T = pd.concat(all_pages, ignore_index=True, sort=False)
T.columns = headers_full

T["nec"] = T["nec"].astype('Int64')
T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)

summary = T.tail(1) #quality control
T=T.iloc[:-1,:]
T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)

T2016 = T.set_index('k')
summary2016 = summary

assert(summary2016["d_done"].values[0] == T2016["d_done"].sum())
assert(summary2016["t_done"].values[0] == T2016["t_done"].sum())
assert(summary2016["d_scheduled"].values[0] == T2016["d_scheduled"].sum())
assert(summary2016["t_scheduled"].values[0] == T2016["t_scheduled"].sum())

T2016[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2016[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2016['total_passed'] = T2016['d_done'].multiply(T2016['d_rate'], fill_value = 1) + T2016['t_done'].multiply(T2016['t_rate'], fill_value =1)
total_passed = T2016['total_passed'].sum() / (T2016["d_done"].sum() + T2016["t_done"].sum())
assert(round(total_passed, 4) == summary2016["total_rate"].values[0])

In [17]:
year=2017
path = files[year]

all_pages = tabula.read_pdf(path, stream=True, pages="all", pandas_options={'header': None})
T = pd.concat(all_pages, ignore_index=True, sort=False)
T.columns = headers_full

T["nec"] = T["nec"].astype('Int64')
T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)

summary = T.tail(1) #quality control
T=T.iloc[:-1,:]
T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)

T2017 = T.set_index('k')
summary2017 = summary
assert(summary2017["d_done"].values[0] == T2017["d_done"].sum())
assert(summary2017["t_done"].values[0] == T2017["t_done"].sum())
assert(summary2017["d_scheduled"].values[0] == T2017["d_scheduled"].sum())
assert(summary2017["t_scheduled"].values[0] == T2017["t_scheduled"].sum())

T2017[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2017[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2017['total_passed'] = T2017['d_done'].multiply(T2017['d_rate'], fill_value = 1) + T2017['t_done'].multiply(T2017['t_rate'], fill_value =1)
total_passed = T2017['total_passed'].sum() / (T2017["d_done"].sum() + T2017["t_done"].sum())
assert(round(total_passed, 4) == summary2017["total_rate"].values[0])

In [18]:
year=2018
path = files[year]
columns_x = [55, 70, 300, 340, 400, 450, 500, 560, 600, 680, 720]
page1 = tabula.read_pdf(path, guess = False, relative_area=True, area=[15, -1, 95, 95], columns=columns_x, pages='1', stream=True, pandas_options={'header': None})
# 2 - 23
rest = tabula.read_pdf(path, guess = False, relative_area=True, area=[9, -1, 95, 95], columns=columns_x, pages='2-23', stream=True, pandas_options={'header': None})

all_pages = sum([page1, rest], [])
T = pd.concat(all_pages, ignore_index=True, sort=False)

T = T.drop([0], axis=1)
T.columns = headers_full

T["nec"] = T["nec"].astype('Int64')
T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)

summary = T.tail(1) #quality control
T=T.iloc[:-1,:]
T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)

T2018 = T.set_index('k')
summary2018 = summary
assert(summary2018["d_done"].values[0] == T2018["d_done"].sum())
assert(summary2018["t_done"].values[0] == T2018["t_done"].sum())
assert(summary2018["d_scheduled"].values[0] == T2018["d_scheduled"].sum())

# missing one exam from summary
# assert(summary2018["t_scheduled"].values[0] == T2018["t_scheduled"].sum())

T2018[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2018[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2018['total_passed'] = T2018['d_done'].multiply(T2018['d_rate'], fill_value = 1) + T2018['t_done'].multiply(T2018['t_rate'], fill_value =1)
total_passed = T2018['total_passed'].sum() / (T2018["d_done"].sum() + T2018["t_done"].sum())
assert(round(total_passed, 4) == summary2018["total_rate"].values[0])

In [19]:
print(T2018["t_scheduled"].sum())
print(summary2018["t_scheduled"].values[0])


190326
190325


In [20]:
year=2019
path = files[year]

columns_x = [85, 600, 800, 885, 1000, 1100, 1200, 1300, 1400, 1550]
page1 = tabula.read_pdf(path, guess = False, relative_area=True, area=[9, 0, 95, 97], columns=columns_x, pages='1', stream=True, pandas_options={'header': None})
# 2 - 21
rest = tabula.read_pdf(path, guess = False, relative_area=True, area=[5, 0, 95, 97], columns=columns_x, pages='2-21', stream=True, pandas_options={'header': None})

all_pages = sum([page1, rest], [])
T = pd.concat(all_pages, ignore_index=True, sort=False)

T.columns = headers_full

T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)


summary = T.tail(1) #quality control
T=T.iloc[:-1,:]
T["nec"] = T["nec"].apply(int)
T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)

T2019 = T.set_index('k')
summary2019 = summary
assert(summary2019["d_done"].values[0] == T2019["d_done"].sum())
assert(summary2019["t_done"].values[0] == T2019["t_done"].sum())
assert(summary2019["d_scheduled"].values[0] == T2019["d_scheduled"].sum())
assert(summary2019["t_scheduled"].values[0] == T2019["t_scheduled"].sum())

T2019[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2019[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2019['total_passed'] = T2019['d_done'].multiply(T2019['d_rate'], fill_value = 1) + T2019['t_done'].multiply(T2019['t_rate'], fill_value =1)
total_passed = T2019['total_passed'].sum() / (T2019["d_done"].sum() + T2019["t_done"].sum())
assert(round(total_passed, 4) == summary2019["total_rate"].values[0])

In [21]:
year=2020
path = files[year]
columns_x = [150, 600, 800, 885, 1000, 1100, 1230, 1300, 1490, 1550]
page1 = tabula.read_pdf(path, guess = False, relative_area=True, area=[10, 0, 95, 97], columns=columns_x, pages='1', stream=True, pandas_options={'header': None})
# 2 - 20
rest = tabula.read_pdf(path, guess = False, relative_area=True, area=[5, 0, 95, 97], columns=columns_x, pages='2-20', stream=True, pandas_options={'header': None})

all_pages = sum([page1, rest], [])
T = pd.concat(all_pages, ignore_index=True, sort=False)

T.columns = headers_full

T["nec"] = T["nec"].astype('Int64')
T['t_rate'] = T['t_rate'].apply(parse_rate)
T['d_rate'] = T['d_rate'].apply(parse_rate)
T["d_done"] = T["d_done"].astype('Int64')
T["d_scheduled"] = T["d_scheduled"].astype('Int64')
T["total_scheduled"] = T["total_scheduled"].astype('Int64')
T["total_done"] = T["total_done"].astype('Int64')
T['total_rate'] = T['total_rate'].apply(parse_rate)
T['k'] = T['name_raw'].apply(name_to_keyword) + "-" + T['nec'].apply(nec_to_string)

T2020 = T.set_index('k')

# # comparing with aggreagates at
# # https://www.imt-ip.pt/sites/IMTT/Portugues/EnsinoConducao/taxasdeaprovacao/Documents/ANO%202020/categorias-2020.pdf
# # differences are maybe to a missing school 
print(T2020["t_scheduled"].sum(), "aggregates: 189918")
assert(T2020["t_done"].sum() == 175301)
print(T2020["d_scheduled"].sum(), "aggregates: 191123")
print(T2020["d_done"].sum(), "aggregates: 177933")

T2020[['d_done', 't_done', 'd_scheduled', 't_scheduled']] = T2020[['d_done', 't_done', 'd_scheduled', 't_scheduled']].fillna(0)

T2020['total_passed'] = T2020['d_done']*T2020['d_rate'] + T2020['t_done']*T2020['t_rate']
T2020['total_passed'] = T2020['d_done'].multiply(T2020['d_rate'], fill_value = 1) + T2020['t_done'].multiply(T2020['t_rate'], fill_value =1)
total_passed = T2020['total_passed'].sum() / (T2020["d_done"].sum() + T2020["t_done"].sum())
print(round(total_passed, 4), "PDF: 0.7018")

189913.0 aggregates: 189918
191114 aggregates: 191123
177924 aggregates: 177933
0.7018 PDF: 0.7018


In [22]:
T = pd.concat([T2015, T2016, T2017, T2018, T2019, T2020], keys=["2015", "2016", "2017", "2018", "2019", "2020"])

In [23]:
m = {'a-desportiva-v-n-gaia-00466': 'a-desportiva-vngaia-00466',
     'academia-boa-viagem-00407': 'boa-viagem-00407',
     'escola-de-conduucao-alto-nivel-de-albergaria-a-velha-00461':'alto-nivel-de-albergaria-a-velha-00461',
     'hugo-vieira-00017': 'hugo-vieira-porto-00017'}

T = T.rename(index=m)

In [24]:
pd.to_pickle(T, "./parsed-data/db.pkl")

In [25]:
import json

In [26]:
result = T.to_json(orient="table")

parsed = json.loads(result)

# json.dumps(parsed, inden2=
with open('./parsed-data/db.json', 'w') as fp:
    fp.write(json.dumps(parsed, indent=4)) 
with open('./parsed-data/db-line.json', 'w') as fp:
    fp.write(json.dumps(parsed)) 